In [ ]:
fruit_first_letters = fruits.map(lambda fruit: fruit[0])
max_fruit_len = fruits.map(lambda fruit: len(fruit)).reduce(lambda a, b: max(a, b), initial=0)
# InvariantNumber(<max_value>, addresses=[<address>])

## Selecting Messages

A `Trace` object can be used to select specific messages from the trace. This is useful for selecting messages that are relevant to the test assertions.

In [5]:
# select the first trace message
trace.messages(0)

{'role': 'user', 'content': 'Hello there'} at 0

In [6]:
# select all user messages
trace.messages(role="user")

InvariantList[{'role': 'user', 'content': 'Hello there'}, {'role': 'user', 'content': 'I need help with something.'}] at [['0'], ['2']]

In [7]:
# select the message with 'something' in the content
trace.messages(content=lambda c: 'something' in c)

InvariantList[{'role': 'user', 'content': 'I need help with something.'}] at [['2']]

On the one hand, the `trace.messages(...)` selector function gives you a convenient way to select messages from the trace. However, in addition to this, it also keeps track of the exact path of the resulting objects in the trace.

This is useful for debugging and backtracking assertion failures, down to the exact agent event that is causing the failure.

Whenever accessing a message, property or sub-dictionary, Invariant Testing will track the exact path and range of its location in the trace.

In [14]:
# selecting content from the 2nd message in the trace
content = trace.messages(2)["content"]
print(content)

I need help with something. at 2.content:0-27


## Selecting Tool Calls

Similar to selecting messages, you can also select just tool calls from the trace.

In [3]:
greet_calls = trace.tool_calls(name="greet")
print(greet_calls[0])

{'type': 'function', 'function': {'name': 'greet', 'arguments': {'name': 'there'}}} at ['1.tool_calls.0']


Again, all accesses to tool calls are tracked and include the exact source path and range in the trace (`1.tool_calls.0` here).

Note that even though you can select `.tool_calls()` directly on `name` and `arguments`, the returned object is always of `{'type': 'function', 'function': { ... }}` shape.

## Other Accessors and Transformers

In addition to accessing different message types, you can also use custom accessors and transformers to compute extra information from specific trace objects:

In [ ]:
# check that the first message is not too far from "Hello there"
trace.messages(0)["content"].levenstein("Hello there") < 0.9

# check that the first message is not too far from "Hello there" in embeddings
trace.messages(0)["content"].embedding_distance("Hello there") < 0.9

# check the length of the resposne
trace.messages(0)["content"].len() < 10

# check the number of tokens
trace.messages(0)["content"].tokens().len() < 10

TODO: extend with more docs on accessors and transformers

Here, member functions are used to access to compute derived values such as Levenshtein distance and tokenization of the text. 

These so-called *transformer* functions on trace objects have the special property that they are fully tracked by the Invariant Testing library: Any result of a transformer function can be tracked back fully to exact input trace objects that were used to compute it.

This is especially helpful to track assertion failures back to the exact source of the problem.

For instance, when comparing `trace.messages(1001)["content"].len() < 10`, the assertion failure can be attributed to the exact (1001th) message content that caused the failure, rather than just failing on the entire trace as a whole.

## Making Assertions

Invariant Testing supports two types of checks:

* Assertions: `assert_that` and `assert_equals`
* Expectations: `expect_that` and `expect_equals`

Assertions are used to strictly check the trace for a specific condition. If the condition is not met, the test case will count as failed.

Expectations are used to check the trace for a specific condition, but the test case will not fail if the condition is not met. This is useful for checking optional conditions or for debugging.

A simple Invariant test case looks like this:

In [ ]:
from invariant_runner.custom_types.assertions import (
    assert_equals,
    assert_that
)

from invariant_runner.custom_types.matchers import is_sentiment

def test_my_property():
    trace = Trace(trace=[...])

    # test that the first assistant message greets the user with 'Hello there'
    assert_equals(trace.messages(role="assistant")[0]["content"], "Hello there")

    # test that the first assistant message has a friendly tone
    assert_that(trace.messages(role="assistant")[0], is_sentiment("friendly"))

Here, we make two assertions:

1. The first assistant message should have the content "Hello there"
2. The first assistant message's sentiment should be "friendly"

For the first assertion, we use a simple equality check. For the second assertion, we use a custom matcher `is_sentiment` to check the sentiment of the message.


## Matchers

Next to exact equality checks, Invariant Testing supports custom matchers to check for more complex conditions.

Matchers are functions that take a trace value or derived value as produced by transformer and return a boolean value indicating whether the condition is met.

For example, above we have made use of the following custom matcher:

```python
assert_that(trace.messages(0)["content"], is_sentiment("friendly"))
```

This matcher checks whether the sentiment of the message is "friendly" and returns `True` if and only if the sentiment is indeed classified as "friendly".